# Extract Sub Grids

Read grid files and generate various subsets.

In [3]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd  # pylint: disable=import-error
# import shutil
# import common

sys.path.insert(0, '../../python')
from simpython.cmg import gridfile  # type: ignore # pylint: disable=import-error,wrong-import-position

In [4]:
import importlib
importlib.reload(gridfile)

<module 'simpython.cmg.gridfile' from '/home/tiago/dev/SimModels/graph_nn/grid/../../python/simpython/cmg/gridfile.py'>

## Options

In [5]:
def list_files(folder_path):
    """ List files names in folder """
    try:
        folder_path = Path(folder_path)
        filenames = [file for file in folder_path.iterdir() if file.is_file()]
        filenames.sort()
        return filenames
    except FileNotFoundError:
        print(f"Folder '{folder_path}' not found.")
        return []

In [10]:
geo_folder_path = '../../Unisim_iv_2024/grid/2024/prior'
geo_folder_path = Path(geo_folder_path)
all_files = list_files(geo_folder_path)

In [11]:
options = {
    'ni': 47,
    'nj': 39,
    'nk': 291,
    'file_names': ['PERMI', 'PERMJ', 'PERMK', 'POR'],
    'subgrids': [],
}

for k in range(41,54):
    options['subgrids'].append( (33,37,12,16,k,k) )
for k in range(75,96):
    options['subgrids'].append( (33,37,12,16,k,k) )
for k in range(206,224):
    options['subgrids'].append( (17,21,19,23,k,k) )
for k in range(229,270):
    options['subgrids'].append( (17,21,19,23,k,k) )

## Main Functions

In [12]:
all_files[81].name

'PERMI_82.geo'

In [13]:
grid = gridfile.GridFile(file_path=all_files[81])

ni = options['ni']
nj = options['nj']
nk = options['nk']

print(grid.get_keyword())
print(grid.get_number_values() == ni*nj*nk)

values = np.array(grid.get_values())
values_3d = values.reshape(nk, nj,-1)

PERMI ALL
True


In [14]:
def n2ijk(n):
    k = int(n/(ni*nj)) + 1
    j = int( (n/ni - (k-1)*nj)) + 1
    i = n - (k-1)*ni*nj - (j-1)*ni + 1
    return i,j,k

def ijk2n(i,j,k):
    return (k-1)*ni*nj + (j-1)*ni + i-1

def get_value(i,j=None,k=None):
    if j is None:
        return values[i]
    elif k is not None:
        return values_3d[k-1,j-1,i-1]
    raise ValueError('Invalid option!')

def get_subgrid(range_, one_dimension=False):
    i1, i2, j1, j2, k1, k2 = range_
    i1 -= 1
    j1 -= 1
    k1 -= 1
    if one_dimension:
        return values_3d[k1:k2, j1:j2, i1:i2].reshape(-1)
    return values_3d[k1:k2, j1:j2, i1:i2]

def write_grid(output_file_path, values):
    gridfile.GridFile.write_(
        output_file_path=output_file_path,
        values=values,
        keyword=grid.get_keyword(),
        comments=grid.get_comments())

In [15]:
write_grid(Path('./test.geo'), get_subgrid(options['subgrids'][0], one_dimension=True))